In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

import warnings

warnings.filterwarnings("ignore")

def mRmR(df, feature_columns, target_column, shuffletime=5, maxpredictors=5,testsize=0.3):
    selected_features = []
    features_left = feature_columns.copy()
    ForPlot=False
    FinalBestAccuracy=float('inf')
    for i in range(1, maxpredictors+1):
        r2_scores = []
        mean_r2_scores = []
        std_r2_scores = []

        rmse_scores = []
        mean_rmse_scores = []
        std_rmse_scores = []
        rmse_scores_train = []
        bestaccuracies = []

        # For the first iteration, evaluate each feature individually
        if i == 1:
            features_to_evaluate = features_left
        # For subsequent iterations, combine the selected features with each remaining feature
        else:
            features_to_evaluate = [selected_features + [feature] for feature in features_left]
        if len(features_to_evaluate)!=0:
            for features in features_to_evaluate:
                print("Currently selected features:",features)
                # If features is a list (combined features), join them for GPR_FOR_Shuffletimes
                if isinstance(features, list):
                    current_features = features
                else:
                    current_features = [features]
    
                # Call GPR_FOR_Shuffletimes with the current feature(s)
                best_rmse, best_r2_for_best_accuracy, mean_rmse, std_rmse, mean_r2, std_r2,best_rmse_train, bestaccuracy = GPR_FOR_Shuffletimes(
                    df, current_features, target_column,ForPlot, test_size=testsize,
                    fixed_length_scale=False, kernel_length_scale=1,
                    shuffle_times=shuffletime
                )
    
                # Store scores
                r2_scores.append(best_r2_for_best_accuracy)
                mean_r2_scores.append(mean_r2)
                std_r2_scores.append(std_r2)
                bestaccuracies.append(bestaccuracy)
    
                rmse_scores.append(best_rmse)
                mean_rmse_scores.append(mean_rmse)
                std_rmse_scores.append(std_rmse)
                rmse_scores_train.append(best_rmse_train)
                
                print(f"R2: {best_r2_for_best_accuracy:.3f}, StdR2Value: {std_r2:.3f}, Accuracy (RMSEtest*(1-R2)): {bestaccuracy:.7f}")
                print(f"Best_RMSE_test: {best_rmse:.4f}, Best_RMSE_train: {best_rmse_train:.4f}, AverageRMSEValue: {mean_rmse:.4f}, StdRMSEValue: {std_rmse:.5f}")
                print("----------------------------------------------")
        # Find the best feature (or combination) based on RMSE
        best_idx = np.argmin(bestaccuracies)

        if FinalBestAccuracy*0.99<bestaccuracies[best_idx]: #The improvements has to be higher than 1 % else ignored
            print("No improvement detected - revert to previous best combination")
            print("----------------------------------------------")
            break
        else:
            best_feature_or_combination = features_to_evaluate[best_idx]

            FinalBestR2=r2_scores[best_idx]
            FinalMeanR2=mean_r2_scores[best_idx]        
            FinalStdR2=std_r2_scores[best_idx]
            
            FinalBestRMSE=rmse_scores[best_idx]
            FinalMeanRMSE=mean_rmse_scores[best_idx]        
            FinalStdRMSE=std_rmse_scores[best_idx]
            FinalBestRMSETrain=rmse_scores_train[best_idx]
            FinalBestAccuracy = bestaccuracies[best_idx]
    
            # Update selected_features and features_left
            if isinstance(best_feature_or_combination, list):
                new_feature = [f for f in best_feature_or_combination if f not in selected_features][0]
                selected_features.append(new_feature)
                features_left.remove(new_feature)
            else:
                selected_features.append(best_feature_or_combination)
                features_left.remove(best_feature_or_combination)

        # Print or store the best feature/combination for this iteration

        print(f"Iteration {i}: Selected {best_feature_or_combination} with Accuracy (RMSEtest*(1-R2)) = {FinalBestAccuracy:.7f}")
        print("----------------------------------------------")

    ForPlot=True

    # Call GPR_FOR_Shuffletimes with the current feature(s)
    best_rmse, best_r2_for_best_accuracy, mean_rmse, std_rmse, mean_r2, std_r2,best_rmse_train, bestaccuracy = GPR_FOR_Shuffletimes(
                df, best_feature_or_combination, target_column,ForPlot, test_size=testsize,
                fixed_length_scale=False, kernel_length_scale=1,
                shuffle_times=shuffletime
            )    
    return selected_features,[FinalBestR2,FinalMeanR2,FinalStdR2],[FinalBestRMSE,FinalMeanRMSE,FinalStdRMSE,FinalBestRMSETrain]

        
def GPR_FOR_Shuffletimes(df, feature_columns, target_column,ForPlot, test_size=0.25, fixed_length_scale=False, kernel_length_scale=1, shuffle_times=5):

    # Extract features and target from DataFrame
    X_final = df[feature_columns].values.astype(float)
    y = df[target_column].values.astype(float)
    
    # Ensure X_final is 2D
    if X_final.ndim == 1:
        X_final = X_final.reshape(-1, 1)

    # Kernel Definition
    optimizer_setting = 'fmin_l_bfgs_b'
    noise_level = 1e-5  # Initial guess for noise variance (adjust as needed)

    if fixed_length_scale:
        # Standardized data requires smaller length scales (e.g., 0.1 to 10)
        kernel = C(1.0, (1e-2, 1e2)) * RBF(length_scale=kernel_length_scale, length_scale_bounds='fixed') + WhiteKernel(noise_level=noise_level, noise_level_bounds=(1e-10, 0.5))
        optimizer_setting = None
    else:
        # Allow length scale to go below 1.0 (down to 1e-2 or 1e-1)
        # Allow ConstantKernel (variance) to be around 1.0 (bounds 1e-2 to 1e2)
        kernel = C(1.0, (1e-2, 1e2)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(noise_level=noise_level, noise_level_bounds=(1e-10, 0.5))
    

    gpr = GaussianProcessRegressor(
        kernel=kernel,
        n_restarts_optimizer=20,
        optimizer=optimizer_setting,
        random_state=42,
        normalize_y=True
    )

    # Standardize X_final
    #X_final, X_m, X_s = standadize(X_final)

    # Initialize lists to store scores
    r2_scores = []
    rmse_scores = []
    rmse_scores_train = []

    # Initialize variables to track the best RMSE and its corresponding R²
    bestaccuracy = float('inf')
    best_r2_for_best_accuracy = None

    # Initialize a variable to store the unscaled training data for the best shuffle

    # Loop over shuffles
    for shuffle in range(shuffle_times):

        # Split the data
        X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
            X_final, y, np.arange(len(df)), test_size=test_size, random_state=None
        )

        # Standardize using only training data
        X_train, X_m, X_s = standadize(X_train)
        X_test = (X_test - X_m) / X_s

        # Train and Predict
        gpr.fit(X_train, y_train)

        # If the kernel length and whitenoise progress should be shown
        #print(f"Optimized Kernel: {gpr.kernel_}")
        
        y_pred_train = gpr.predict(X_train)
        y_pred_test = gpr.predict(X_test)

        # Calculate Metrics
        rmsetest = np.sqrt(mean_squared_error(y_test, y_pred_test))
        rmsetrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
        r2 = r2_score(y_test, y_pred_test)
        current_accuracy = rmsetest * (1 - r2) 

        # Store scores
        r2_scores.append(r2)
        rmse_scores.append(rmsetest)
        rmse_scores_train.append(rmsetrain)

        
        # Update best RMSE and corresponding results
        if current_accuracy < bestaccuracy:
            bestaccuracy = current_accuracy
            best_rmse = rmsetest
            best_rmse_train = rmsetrain
            best_r2_for_best_accuracy = r2
            best_y_train, best_y_pred_train, best_y_test, best_y_pred_test = y_train, y_pred_train, y_test, y_pred_test

    # After all shuffles, plot the best result
    if ForPlot and best_y_train is not None:
        plot_train_test_scatter(best_y_train, best_y_pred_train, best_y_test, best_y_pred_test,target_column)
        print(f"Plotted results for shuffle with best accuracy, RMSE_test={best_rmse:.4f}, R²={best_r2_for_best_accuracy:.4f}), Accuracy (rmsetest*(1-R2)) = {bestaccuracy:.7f}")
        
    # Calculate cross-validation scores (mean and std)
    mean_rmse = np.mean(rmse_scores)
    std_rmse = np.std(rmse_scores)
    mean_r2 = np.mean(r2_scores)
    std_r2 = np.std(r2_scores)

    return best_rmse, best_r2_for_best_accuracy, mean_rmse, std_rmse, mean_r2, std_r2,best_rmse_train, bestaccuracy

def standadize(X):
    X = np.array(X)
    X_m = np.mean(X, axis=0)
    X_s = np.std(X, axis=0)
    X -= X_m
    X /= X_s
    return X, X_m, X_s  # Return mean and std for unstandardization



def plot_train_test_scatter(X_train, y_train, X_test, y_pred,target_column):
    """
    Plots training data and test predictions as a scatter plot.

    Parameters:
    - X_train: array-like, training feature values
    - y_train: array-like, training target values
    - X_test: array-like, test feature values
    - y_pred: array-like, predicted target values for test data
    - title: str, title of the plot (default: 'Scatter Plot: Training vs. Test Predictions')
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(X_train, y_train, color='blue', label='Training Data')
    plt.scatter(X_test, y_pred, color='red', label='Test Predictions')

    plt.xlabel("Experimental " + target_column)
    plt.ylabel("Predicted " + target_column)
    #plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

##############################################################
############## targets examples
#  IV :  "PCE_IV","VOC_IV","JSC_IV","FF_IV"

#  Spectral features: 'features_A.1.c_UV','features_A.1.w_UV','features_A.1a.a_UV','features_A.1a.c_UV','features_ADRatio','features_A_UV','features_Atot_UV',
# 'features_D.1.c_UV', 'features_D.1.w_UV', 'features_D.1a.a_UV', 'features_D.1a.c_UV', 'features_D_UV', 'features_Dtot_UV', 'features_XA_UV', 'features_XD_UV',
# 'features_XamA_UV', 'features_XamD_UV', 'features_atot_UV'

target_column = "VOC_IV"

############## features examples
# There are two feature sets. They can be selected with
# plan parameters:Features = "plan_"
#Features = "plan_"

# spectral features: Features = "features_"
Features = "features_"

############## Read in data
current_dir = os.getcwd()  # Gets the current working directory
file_name = "ML - Raw data - Optimization.csv"

df = pd.read_csv(os.path.join(current_dir, file_name))

# Extracts all columns beginning with string defined in "Features"
feature_columns = [col for col in df.columns if col.startswith(Features)]


############## Set shuffle times and max predictors

selected,ListResultR2,ListResultRMSE = mRmR(df, feature_columns, target_column,shuffletime=20, maxpredictors=3)

##############################################################

# Output

print("Selected features:", selected)
print(f"BestR2Value: {ListResultR2[0]:.3f}, AverageR2Value: {ListResultR2[1]:.3f}, StdR2Value: {ListResultR2[2]:.4f}")
print(f"Best_RMSE_test: {ListResultRMSE[0]:.4f}, Best_RMSE_train: {ListResultRMSE[3]:.4f}, AverageRMSEValue: {ListResultRMSE[1]:.4f}, StdRMSEValue: {ListResultRMSE[2]:.5f}")
